### ASSIST tests: (3666) Holman term-by-term comparison

Here we carry out a term-by-term comparison of the components of the various effects that contribute to the acceleration of asteroid (3666) Holman.  This comparison is between the results of ASSIST and those of the JPL small bodies integrator.

ASSIST is a set of python-wrapped C functions that implement an ephemeris quality-integrator that have been compiled into a library.  That library is imported with the assist.py package.

The primary function in assist.py is 'integration_function', which integrates massless test particles in the field of the Sun, planets, Moon, and 16 massive asteroids.  It also includes the J2, J3, and J4 gravitational harmonics of the Earth, the J2 gravitational harmonic of the Sun, and the solar GR terms (using the PPN formulation).  

The positions of the massive bodies come from two binary files, both from JPL.  The first is for the Sun, planets, and Moon, with the latest DE441 ephemeris. The other is for the asteroids, corresponding to DE441.  

The coordinate frame and units are not flexible, currently.  The coordinate frame is the equatorial ICRF, which is the native coordinate system for the JPL binary files.  Note that this is equatorial rather than ecliptic.  In addition, the native coordinates are barycentric, rather than heliocentric. 

For units we use solar masses, au, and days.  The independent time coordinate is TDB in Julian days.

The underlying numerical integrator is IAS15 (Rein & Liu 2015), a 15th order predictor-corrector integrator with an adaptive step-size.  Each time step involves eight sub-steps.  We have modified the integrator to output the state at each of the sub-steps in order to support interpolation of the output.

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import ctypes
import importlib

import spiceypy as spice

import pkg_resources
import sys
sys.path.append(r'/Users/mholman/assist')
import assist

jd_ref = spice.j2000()

plt.rc('text', usetex = True)


In [2]:
# This cell defines the initial conditions we have used for this test.
# The dynamical state available on Horizons now will be slightly different because 
# additional observations have been incorporated into the orbital solution.
#
# (3666) Holman at 2458849.5 TDB
# From email with Davide Farnocchia
row = [3.3388753502614090e+00, -9.1765182678903168e-01, -5.0385906775843303e-01,
     2.8056633153049852e-03,  7.5504086883996860e-03,  2.9800282074358684e-03]

In [3]:
%%time
# Here we are looking at the term-by-term acceleration without actually integrating.
# We simply evaluate the acceleration of the test particle at the start of a zero length
# integration

# We output both GR calculations for this comparison, but we only accumulate one of them.

# Full GR
# direct n-body accelerations, computed in ephemeris order.
# Earth J2, J3, and J4
# Solar J2

# This generates three files: acc.out, eih_acc.out, and var_acc.out. 

# acc.out: a tabulation of the various acceleration terms, body by body,
# along with the Earth J2-J3-J4, Solar J2, and the two forms of GR used
# in the JPL integrator.

# eih_acc.out: a break out of the constituent contributions of the full GR.  
# We provide the equation in the paper, so we could point to the different 
# terms, but the main point is that the agreement is good to machine precision.
# Davide provided this to support our debugging, so that we could demonstrate
# that all the acceleration components had been coded correctly.  We use
# JPL's ephemeris code as basis of comparison because has been thoroughly
# tested through decades of use in spacecraft trajectory calculations. 
# Furthermore, matching the results of Horizons is a primary objective.

# var_acc.out: this a comparison of the analytic and numerical partial derivatives
# (computed with pairs of real particles with displaced initial conditions)
# of the acceleration, component by component.  

tstart, tstep, trange = 2458849.5-jd_ref, 20.0, 0

epoch = tstart
tend = tstart + trange

instates = np.array([row])
n_particles = 1

part_params = None
#part_params = np.array([[1e-10, 0.0, 0.0],[2e-10, 0.0, 0.0]])
#part_params = np.array([[0.0, 1e-10, 0.0],[0.0, 2e-10, 0.0]])
#part_params = np.array([[0.0, 0.0, 1e-10],[0.0, 0.0, 2e-10]])
geocentric = 0

n_var = 1

invar_part = np.zeros(0, dtype=int)
invar = np.zeros((1, 6))
var_part_params = np.array([1., 0., 0.])
#var_part_params = np.array([0., 1., 0.])
#var_part_params = np.array([0., 0., 1.])

nsubsteps = 10
hg = np.arange(0, 1.1, 0.1, dtype=np.double)

times, states, var, var_ng, status = assist.integration_function(tstart, tend, tstep, geocentric, 
                                                                       n_particles, instates, part_params, n_var, invar_part, invar, var_part_params, hg,
                                                                       nsubsteps=nsubsteps)

#(states[:,1,:]-states[:,0,:])[-1]-(var*1e-10)[-1]

CPU times: user 1.84 ms, sys: 2.75 ms, total: 4.59 ms
Wall time: 2.88 ms


In [1]:
states[-1]

NameError: name 'states' is not defined